# Imports and Models

In [86]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain import PromptTemplate
import langchain
import openai

from pypdf import PdfReader

import pinecone

import os
from dotenv import load_dotenv, find_dotenv

find_dotenv()
load_dotenv(find_dotenv(), override=True)
openai.api_key=os.getenv("OPENAI_KEY")

In [127]:
llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_KEY"), model='gpt-4')
embedding_generator = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_KEY"))

# Reading and chunking the document

In [48]:
file = '../data/itau2t23.pdf'

reader = PdfReader(file)
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1000,
    chunk_overlap=200)

chunks = text_splitter.create_documents(texts=[text])

# Creating the vector DB

In [51]:
# Initializing Pinecone Vector DB
pinecone.init(      
	api_key=os.getenv("PINECONE_KEY"),      
	environment=os.getenv("PINECONE_ENV")      
)  
index = pinecone.Index(os.getenv("PINECONE_INDEX")) 

In [52]:
# create new embedding to upsert in vector store
doc_db = Pinecone.from_documents(
          chunks,
          embedding_generator,
          index_name=os.getenv("PINECONE_INDEX")
        )

## In case we already have our indexes stored, just use it

In [65]:
# doc_db = Pinecone.from_existing_index(os.getenv("PINECONE_INDEX"), embedding_generator)

# Using GPT with RAG

In [81]:
prompt_template_wo_rag = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: {question}
=========

=========
FINAL ANSWER:"""

In [84]:
QUERY = 'Qual foi o indice de eficiência do Itaú no segundo trimestre de 2023?'

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'Você é um assistente que responde em Portugûes do Brasil'},
        {'role': 'user', 'content': prompt_template_wo_rag.format(question=QUERY)}
    ],
    model='gpt-4'
)

response = response.choices[0].message.content

print(response)

Desculpe, mas as informações fornecidas não contêm dados sobre o índice de eficiência do Itaú no segundo trimestre de 2023.

FONTE: Informações fornecidas.


ChatGPT **can't** answer because it doesn't have the information!

In [78]:
# prompt template taken from: https://github.com/smatiolids/astra-agent-memory/tree/main

prompt_template = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["summaries", "question"]
)

In [79]:
# Create a "RetrievalQA" chain
langchain.verbose = False
chainSim = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=doc_db.as_retriever(),
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)

In [59]:
QUERY = 'Qual foi o indice de eficiência do Itaú no segundo trimestre de 2023?'

# Run it and print results
responseSim = chainSim.run(QUERY)
print(responseSim)


No segundo trimestre de 2023, o índice de eficiência do Itaú foi de 39,6%. SOURCES: Transcrição da conferência de resultados do Itaú Unibanco do segundo trimestre de 2023.


In [64]:
QUERY = 'Qual foi o custo de crédito do Itaú no trimestre?'

# Run it and print results
responseSim = chainSim.run(QUERY)
print(responseSim)


O custo de crédito do Itaú no trimestre foi de R$9,4 Bilhões, com um índice de custo de crédito sobre carteira de 3,3%. Além disso, o índice de cobertura veio em 212%, mantendo-se estável.

SOURCES:
- https://www.itau.com.br/investidores/resultados/resultados-trimestrais/


## Using OpenAI API call and DB query

In [122]:
QUERY = 'Qual foi o indice de eficiência do Itaú no segundo trimestre de 2023?'

result_db_retrieval = index.query(
  vector=[embedding_generator.embed_query(QUERY)],
  top_k=4,
  include_values=True,
  include_metadata=True
)
context = [{"text": row['metadata']['text']} for row in result_db_retrieval['matches']]

In [123]:
prompt_template_wo_rag_gpt = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: {question}
=========
CONTEXT: {context}
=========
FINAL ANSWER:"""

In [128]:
QUERY = 'Qual foi o indice de eficiência do Itaú no segundo trimestre de 2023?'

response = openai.ChatCompletion.create(
    messages=[
        {'role': 'system', 'content': 'Você é um assistente que responde em Portugûes do Brasil'},
        {"role": "user", "content": prompt_template_wo_rag_gpt.format(question=QUERY, context=context)}
    ],
    model='gpt-4'
)

response = response.choices[0].message.content

print(response)

O índice de eficiência do Itaú no segundo trimestre de 2023 foi de 39,6%. 

Fontes: Contexto do documento fornecido.
